In [22]:
import numpy as np
import pandas as pd
import math 
import os
import lvm_read
import pickle

In [23]:
#obtain lvm data
#code from this: https://github.com/openmodal/lvm_read/blob/master/Showcase%20lvm_read.ipynb
os.chdir("C:\\Github Repositories\\LabView-Garmin-To-CSV")
LVMfilename = 'test_22-10-11_1349_011.lvm'
lvm = lvm_read.read(LVMfilename,read_from_pickle=False)

In [24]:
lvm[0]

{'Channels': 1,
 'Samples': [10000, nan],
 'Date': ['2022/10/11', ''],
 'Time': ['13:49:49.6749649499425851751', ''],
 'Y_Unit_Label': ['Volts', ''],
 'X_Dimension': ['Time', ''],
 'X0': [0.0, nan],
 'Delta_X': [0.0005, nan],
 'data': array([[8.761779],
        [8.756186],
        [8.753224],
        ...,
        [8.759476],
        [8.754211],
        [8.74796 ]]),
 'Channel names': ['Voltage', 'Comment']}

In [26]:
#organize data for lvm
startTime = lvm[0]['Time'][0]

In [25]:
#coding example for HDF5 https://www.geeksforgeeks.org/how-to-add-metadata-to-a-dataframe-or-series-with-pandas-in-python/

